# OR Tools for Travelling Salesman Problem (TSP)

In [13]:
import random as rd
import numpy as np
import pandas as pd
import math

import time
import datetime

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import matplotlib.pyplot as plt
import seaborn as sns

### 1. 定義參數

In [61]:
NUM_NODES = 16
MAX_X, MAX_Y = 20, 20
NUM_VEHICLES = 1
NUM_DEPOT = 0

### 2. 建立資料

In [67]:
def create_data_model():
    data = {}
    data['locations'] = [(int(np.random.uniform(0, MAX_X)), int(np.random.uniform(0, MAX_Y))) for i in range(NUM_NODES)]
    data['locations'][0] = (int(MAX_X/2), int(MAX_Y/2))
    data["distance_matrix"] = compute_manhattan_distance_matrix(data['locations'])
    data["num_vehicles"] = NUM_VEHICLES
    data["depot"] = NUM_DEPOT
    return data

def compute_manhattan_distance_matrix(nodes): # 取得曼哈頓距離距離矩陣
    distance_matrix = []
    for node_i in nodes:
        distances = []
        for node_j in nodes:
            if node_i == node_j:
                distances.append(0)
            else:
                manhattan_distance = np.sum(np.abs(np.array(node_i) - np.array(node_j)))
                distances.append(manhattan_distance)
        distance_matrix.append(distances)
    return distance_matrix

def compute_euclidean_distance_matrix(locations): # 計算歐式距離距離矩陣
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = int(
                    math.hypot((from_node[0] - to_node[0]), (from_node[1] - to_node[1]))
                )
    return distances

data = create_data_model()
pd.DataFrame(data).head()

,locations,distance_matrix,num_vehicles,depot
0,"(10, 10)","[0, 4, 7, 5, 16, 8, 9, 10, 14, 5, 5, 8, 16, 9,...",1,0
1,"(10, 14)","[4, 0, 3, 9, 20, 12, 13, 14, 18, 1, 9, 12, 20,...",1,0
2,"(8, 15)","[7, 3, 0, 12, 19, 15, 12, 13, 21, 2, 12, 15, 1...",1,0
3,"(14, 9)","[5, 9, 12, 0, 19, 3, 12, 13, 9, 10, 4, 5, 19, ...",1,0
4,"(1, 3)","[16, 20, 19, 19, 0, 20, 7, 6, 16, 21, 15, 24, ...",1,0


In [68]:
data = {}

data["locations"] = [
(456, 320), # location 0 - the depot
(228, 0),    # location 1
(912, 0),    # location 2
(0, 80),     # location 3
(114, 80),   # location 4
(570, 160),  # location 5
(798, 160),  # location 6
(342, 240),  # location 7
(684, 240),  # location 8
(570, 400),  # location 9
(912, 400),  # location 10
(114, 480),  # location 11
(228, 480),  # location 12
(342, 560),  # location 13
(684, 560),  # location 14
(0, 640),    # location 15
(798, 640)   # location 16
]  

data["distance_matrix"] = [
    [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
    [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
    [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
    [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
    [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
    [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
    [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
    [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
    [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
    [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
    [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
    [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
    [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
    [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
    [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
    [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
    [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]
]

data["num_vehicles"] = 1
data["depot"] = 0

### 3. 建立傳送模型

- 程式主要部分中的下列程式碼會建立索引管理員 (manager) 和轉送模型 (routing)。

- `manager.IndexToNode` 方法會將解題工具的內部索引 (您可以放心忽略) 轉換為位置的數字。位置數字對應距離矩陣的索引。

- 若路線的起始點不是 Depot，則需設定 start 與 end, 可參考以下文章: https://developers.google.com/optimization/routing/routing_tasks?hl=zh-tw#setting_start_and_end_locations_for_routes

In [69]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"

def get_routes(solution, routing, manager):
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

def main():
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["depot"])
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index) # Define cost of each arc.

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT
    )
    search_parameters.solution_limit = 100
    search_parameters.time_limit.seconds = 5
    # search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(manager, routing, solution)
        routes = get_routes(solution, routing, manager)
    else:
        print('No solution')

if __name__ == "__main__":
    main()

Objective: 4384 miles
Route for vehicle 0:
 0 -> 9 -> 5 -> 8 -> 6 -> 2 -> 10 -> 16 -> 14 -> 13 -> 12 -> 11 -> 15 -> 3 -> 4 -> 1 -> 7 -> 0



由於 Routing Solver 並不總是向 TSP 返回最佳解，因為路徑問題在計算上很難處理。

為了找到更好的解決方案，您可以使用更高級的搜尋策略，稱為 Guided Local Search ，它使 Solver 能夠逃脫 Local Minimum - 比所有附近路線都短的解決方案，但不是 Global Minimum。遠離 Local Minimum 後，Solver 繼續搜尋答案。

更多搜尋方法: https://developers.google.com/optimization/routing/routing_options

In [22]:
# guided local search
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds = 30
search_parameters.log_search = True

"""
When using GUIDED_LOCAL_SEARCH or other metaheuristics, 
you need to set a time limit for the solver 
— otherwise the solver will not terminate. 
The preceding code sets a time limit of 30 seconds.
"""

'\nWhen using GUIDED_LOCAL_SEARCH or other metaheuristics, \nyou need to set a time limit for the solver \n— otherwise the solver will not terminate. \nThe preceding code sets a time limit of 30 seconds.\n'

由於 Routing Solver 是處理整數，因此，如果距離矩陣包含非整數項目，您就必須將距離四捨五入為整數。如果有些差異比較小，則四捨五入會影響解決方案。

如要避免四捨五入的問題，您可以 scale 距離矩陣：將矩陣的所有項目乘以 100。這會讓任何路徑的長度乘以 100 倍，但不會改變解決方案。這種做法的好處是，當您將矩陣項目四捨五入時，與距離相差的小數值 (最多為 0.5) 是很小的，因此對解決方案沒有太大的影響。

縮放距離矩陣時，您也必須變更 Solution Printer，將縮放後的路徑長度除以縮放比例係數，這樣才能顯示該路徑未縮放的距離。